- Al descargarnos los activos, podemos ver cómo todos ellos tienen el mismo número de variables (columnas), pero cotizaciones distintas (filas). El siguiente objetivo es conseguir homogeneizar todos los datos. Las cotizaciones de cada activo deben de estar en las mismas líneas para poder realizar cálculos. 
- Objetivo: Homogeneiza los datos del benchmark y los activos con una ventana de 500 días.
- Tiempo objetivo: 45 minutos

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
from datetime import timedelta
from time import mktime
import time
from tqdm import tqdm
import math
from rcurl import get_curl
from io import BytesIO

In [2]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance  
    parameters: stock - short-handle identifier of the company    
    returns a tuple of header, crumb and cookie
    """   
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }  
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
        output=(header, crumb[0], website.cookies)
        return output       

In [3]:
def convert_to_unix(date):
    """
    converts date to unix timestamp    
    parameters: date - in format (yyyy-mm-dd)    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

In [4]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format    
    parameters: 
        stock - short-handle identifier of the company        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    
    error1='404 Not Found: Timestamp data missing.'
    
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)   
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        
        return website.text.split('\n')

In [5]:
def set_dates(x,ventana):
    
    """setup inicial de fechas,
    esta funcion devuelve la fecha de inicio
    y fin en string y formato YYYY-MM-DD
    Parametros:
    x=start o end
    ventana: entero referido al numero de dias del periodo"""
    
    from datetime import timedelta
    from datetime import datetime
    hoy=datetime.now()
    if x=='end':
        fecha_fin = str(hoy.now())
        fecha_fin = fecha_fin[0:10]
        return (fecha_fin)
    if x=='start':
        fecha_inicial=hoy-timedelta(ventana)
        fecha_inicial = str(fecha_inicial)
        fecha_inicial = fecha_inicial[0:10]
        return (fecha_inicial)
    else:
        print('inputs incorrectos. Ver docstring d ela funcion')    
    

In [6]:
def intentos_descarga_precios(stock_series,interval,fecha_inicial,fecha_fin, index_series, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series)
        codigo = 'OK'
        e = 'OK'
        return df, codigo, e
    
    except Exception as e:
        
        print('El activo no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        
        return df, codigo, e

In [7]:
def historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series):
    
    n=0
    x=0 
    for i in range(len(stock_series)):
        print("descargando "+stock_series[i]+' del indice '+index_series[i])
        try:
            DF=load_csv_data(stock_series[i], interval=interval, day_begin=fecha_inicial, day_end=fecha_fin)
            DF=pd.DataFrame(DF)
            DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
            DF.columns=DF.iloc[0,:]
            DF=DF.iloc[1:DF.shape[0],:]
            x=DF.shape[1]                 
            if x<2:
                 print("No nos hemos podido descargar el activo", benchmark)
            else:
                DF['stock'] = str(stock_series[i])
                DF['indice'] = str(index_series[i]) 
                DF=DF.dropna()
            
                if n==0:
                    dfacum=DF
                else:
                    dfacum=agrega_dataframes(DF,dfacum)    
                n=n+1
        except Exception as e:
            print("No nos hemos podido descargar el activo")
            print('key error:', e)
            next
        
    return dfacum


In [8]:
def agrega_dataframes(df,dfacum):
    """codigo que permite ir concatenando
    dataframes con estructura similar
    df:daframe a agregar
    dfacum: dataframe en donde se agregara"""
    
    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
    dfacum.reset_index(drop=True)
    return dfacum


In [9]:
def split_df_acum_by_stock(dfacum):
    """codigo que permite ir desconcatenando
    dataframes con estructura similar a partir de un df acumulado
   requiere que la clave de desagregacion sea la columna 'stock'
    dfacum: dataframe en se ha agregado"""
    activos_a_importar = dfacum.stock.unique()
    for activo in activos_a_importar:
        globals()[activo] = dfacum[dfacum['stock']==activo]

In [22]:
def vector_lab_dates(fecha_inicial,ventana):
    """con esta funcion se obtiene un listado de fechas
    entre lunes y viernes, a partir de una fecha inicial
    y una ventana"""
    dates = pd.date_range(fecha_inicial, periods=ventana, freq='B')    
    inicio = datetime.strptime(fecha_inicial, '%Y-%m-%d')
    dias =ventana
    dates=[]
    
    for days in range(dias):
        date = inicio + timedelta(days=days)
        if date.weekday() < 5:
            dates.append(date)
    return pd.DataFrame({'dates':dates})

In [11]:
def componentes_indice_ind_download_0(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    lista_dato = []
    
    #primero se obtiene la divisa
    url="https://es.finance.yahoo.com/quote/"+ticker+"/components/"
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find('span', attrs={'data-reactid': '4'})
    name = name_box.text.strip()
    divisa=name[len(name)-3:len(name)]
    
    print(ticker,divisa)
    
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find_all('td')
    name_boxccc=name_box[0]
    name_empresa=name_box[1]
    name_boxccc = name_boxccc.get_text(strip=True)
    name_empresa = name_empresa.get_text(strip=True)
    
    for i in range(0,len(name_box)):
        name_boxccc=name_box[i]
        name_boxccc = name_boxccc.get_text(strip=True)
        lista_dato.append(name_boxccc)
        
    df=pd.DataFrame ({'dato':lista_dato})
    df['Divisa'] = divisa
    
    return df

In [12]:
def intentos_descarga_indice(ticker, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = componentes_indice_ind_download_0(ticker)
        codigo = 'OK'
        e = 'OK'
        print('El índice ', ticker,' se ha descargado correctamente')
        return df, codigo, e
    
    except Exception as e:
        
        print('El índice ', ticker,' no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        return df, codigo, e

In [13]:
def componentes_indice_ind_download_1(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    intentos = 1
    for i in range(3):
        intentos_descarga_indice_res = intentos_descarga_indice(ticker, intentos)
        df = intentos_descarga_indice_res[0]
        codigo = intentos_descarga_indice_res[1]
        e = intentos_descarga_indice_res[2]
        
        if codigo == 'OK':
            return df
        if (codigo == 'ERROR') & (intentos <=3):
            intentos = intentos + 1
            time.sleep(3)
        if intentos ==3:
            print('Mensaje de error: ',str(e))
            print("Tras varios intentos no nos hemos podido descargar",ticker,"Dejamos de intentarlo.")
            df = pd.DataFrame ({'dato':[],
                                'Divisa':[]})
            return df

In [14]:
def recupera_valor_secuencia(df, start, freq, encabezado):
    valor = pd.DataFrame(np.arange(start, len(df), freq))
    valor['key'] = 1
    valor.index = valor[0]
    valor = valor.drop(0, 1)
    valor = pd.merge(df, valor, left_index=True, right_index=True)
    valor.reset_index(drop=True, inplace=True)
    valor.rename(columns={"dato": encabezado},inplace=True)
    valor = valor.drop(['key'], 1)
    return valor

In [15]:
def componentes_indice_ind_download_2(tickers_de_indices):
    """En este tercer paso, con la informacion descargada,reconstruimos la tabla. 
    Tiene 6 columnas (simbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
    # Nos interesa obtener el símbolo y el nombre de la empresa.
    # Ojo, queremos importar únicamente los activos que tengan cotización. 
    """
    intentos = 0
    
    dfacum = pd.DataFrame ({'Activo':[],
                            'Divisa':[],
                            'Empresa':[],
                            'Precio':[],
                            'Indice':[]})
    
    for indice in tickers_de_indices:
        df = componentes_indice_ind_download_1(indice)
        Activo = recupera_valor_secuencia(df, 0, 6, 'Activo')
        df = df.drop(['Divisa'],1)
        Empresa = recupera_valor_secuencia(df, 1, 6, 'Empresa')
        Precio = recupera_valor_secuencia(df, 2, 6, 'Precio')
        df = pd.merge(Activo, Empresa, left_index=True, right_index=True)
        df = pd.merge(df, Precio, left_index=True, right_index=True)
        df['Indice'] = indice
        df = df[df.loc[:,'Precio'] != '']
        dfacum = agrega_dataframes(dfacum, df)
        dfacum.reset_index(drop=True, inplace=True)
        
    return dfacum

In [16]:
tickers_de_indices=("%5EBFX","%5EBVSP","%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", 
                      "%5EMXX","%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI","%5EJKSE")
ventana = 500

In [17]:
dfacum = componentes_indice_ind_download_2(tickers_de_indices)


%5EBFX EUR
El índice  %5EBFX  se ha descargado correctamente
%5EBVSP BRL
El índice  %5EBVSP  se ha descargado correctamente
%5EDJI USD
El índice  %5EDJI  se ha descargado correctamente
%5EFCHI EUR
El índice  %5EFCHI  se ha descargado correctamente
%5EFTSE GBP
El índice  %5EFTSE  se ha descargado correctamente
%5EGDAXI EUR
El índice  %5EGDAXI  se ha descargado correctamente
%5EHSI HKD
El índice  %5EHSI  se ha descargado correctamente
%5EIBEX EUR
El índice  %5EIBEX  se ha descargado correctamente
%5EMXX MXN
El índice  %5EMXX  se ha descargado correctamente
%5EMERV  en
El índice  %5EMERV  se ha descargado correctamente
%5EOMXSPI SEK
El índice  %5EOMXSPI  se ha descargado correctamente
%5EOSEAX NOK
El índice  %5EOSEAX  se ha descargado correctamente
%5ESSMI CHF
El índice  %5ESSMI  se ha descargado correctamente
%5ESTI SGD
El índice  %5ESTI  se ha descargado correctamente
%5EJKSE IDR
El índice  %5EJKSE  se ha descargado correctamente


In [18]:
fecha_fin=set_dates('end',ventana)
fecha_inicial=set_dates('start',ventana)

print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)

la fecha de inicio es 2019-02-27  y la de fin es 2020-07-11


incluimos el tratamento de fechas del enunciado

In [23]:
dates_df = vector_lab_dates(fecha_inicial, ventana)

In [24]:
dfacum = historic_prices_series(dfacum['Activo'],
                                            '1d', 
                                            fecha_inicial, 
                                            fecha_fin, 
                                            dfacum['Indice'])

descargando KDSI.JK del indice %5EJKSE
descargando BABP.JK del indice %5EJKSE
descargando PSKT.JK del indice %5EJKSE
descargando FMII.JK del indice %5EJKSE
descargando LPGI.JK del indice %5EJKSE
descargando VOKS.JK del indice %5EJKSE


KeyboardInterrupt: 

In [41]:
dfacum

,Adj Close,Close,Date,High,Low,Open,Volume,indice,stock
1,2.514614,2.660000,2019-02-21,2.680000,2.610000,2.670000,5130200,%5ESTI,U96.SI
2,2.524068,2.670000,2019-02-22,2.700000,2.640000,2.650000,4012500,%5ESTI,U96.SI
3,2.505161,2.650000,2019-02-25,2.690000,2.620000,2.680000,5283700,%5ESTI,U96.SI
4,2.495708,2.640000,2019-02-26,2.640000,2.610000,2.640000,3553300,%5ESTI,U96.SI
5,2.486254,2.630000,2019-02-27,2.690000,2.630000,2.640000,3781500,%5ESTI,U96.SI
...,...,...,...,...,...,...,...,...,...
344,12.960000,12.960000,2020-06-29,13.100000,12.540000,12.880000,192111,%5EBFX,ONTEX.BR
345,13.030000,13.030000,2020-06-30,13.200000,12.770000,12.990000,179916,%5EBFX,ONTEX.BR
346,13.050000,13.050000,2020-07-01,13.230000,12.730000,13.100000,302787,%5EBFX,ONTEX.BR
347,13.070000,13.070000,2020-07-02,13.310000,12.870000,13.180000,148783,%5EBFX,ONTEX.BR


In [42]:
# nos aseguramos que nos descargamos todos los registros por activo
muestra_stock = dfacum.groupby(['stock']).count()
muestra_stock

,Adj Close,Close,Date,High,Low,Open,Volume,indice
stock,,,,,,,,
0002.HK,337,337,337,337,337,337,337,337
0003.HK,337,337,337,337,337,337,337,337
0006.HK,337,337,337,337,337,337,337,337
0011.HK,337,337,337,337,337,337,337,337
0012.HK,337,337,337,337,337,337,337,337
...,...,...,...,...,...,...,...,...
Y92.SI,344,344,344,344,344,344,344,344
YPFD.BA,331,331,331,331,331,331,331,331
Z74.SI,344,344,344,344,344,344,344,344


Finalmente, obtenemos un DataFrame por activo, podiamos haberlo hecho antes, pero asi nos aseguramos que todos los activos estan homogeneizados en esta primera fase.

In [45]:
split_df_acum_by_stock(dfacum)

Recuperamos un ejemplo para comprobar que está todo bien

In [48]:
datos = globals().get('0003.HK')
datos.head()

,Adj Close,Close,Date,High,Low,Open,Volume,indice,stock
1,15.705014,16.290899,2019-02-21,16.345501,16.181801,16.200001,16067877,%5EHSI,0003.HK
2,15.757652,16.345501,2019-02-22,16.345501,16.236401,16.345501,12132530,%5EHSI,0003.HK
3,15.599840,16.181801,2019-02-25,16.345501,16.090900,16.345501,22898645,%5EHSI,0003.HK
4,15.617384,16.200001,2019-02-26,16.272699,16.018200,16.200001,14492054,%5EHSI,0003.HK
5,15.775101,16.363600,2019-02-27,16.363600,16.290899,16.345501,17628449,%5EHSI,0003.HK
